In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

C:\Users\aleaz\AppData\Local\Temp\ipykernel_17736\1041642270.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# carga y lectura
datos = pd.read_csv("league_of_lol.csv")

# for each column, get the index and the name
i = 0
for col in datos.columns:
    print("", i, "\t", col)
    i += 1

 0 	 gameId
 0 	 blueWins
 0 	 blueWardsPlaced
 0 	 blueWardsDestroyed
 0 	 blueFirstBlood
 0 	 blueKills
 0 	 blueDeaths
 0 	 blueAssists
 0 	 blueEliteMonsters
 0 	 blueDragons
 0 	 blueHeralds
 0 	 blueTowersDestroyed
 0 	 blueTotalGold
 0 	 blueAvgLevel
 0 	 blueTotalExperience
 0 	 blueTotalMinionsKilled
 0 	 blueTotalJungleMinionsKilled
 0 	 blueGoldDiff
 0 	 blueExperienceDiff
 0 	 blueCSPerMin
 0 	 blueGoldPerMin
 0 	 redWardsPlaced
 0 	 redWardsDestroyed
 0 	 redFirstBlood
 0 	 redKills
 0 	 redDeaths
 0 	 redAssists
 0 	 redEliteMonsters
 0 	 redDragons
 0 	 redHeralds
 0 	 redTowersDestroyed
 0 	 redTotalGold
 0 	 redAvgLevel
 0 	 redTotalExperience
 0 	 redTotalMinionsKilled
 0 	 redTotalJungleMinionsKilled
 0 	 redGoldDiff
 0 	 redExperienceDiff
 0 	 redCSPerMin
 0 	 redGoldPerMin


In [4]:
df.head()

NameError: name 'df' is not defined